## 4.1 PydanticOutputParser

<div style="text-align: right"> Initial issue : 2025.05.02 </div>
<div style="text-align: right"> last update : 2025.05.02 </div>

Langchain의 출력 파서는 LLM의 출력을 더 유용하고 구조화된 형태로 변환하는 컴포넌트임
### 출력 파서의 역할
- LLM의 출력을 받아 더 적합한 형식으로 변환
- 구조화된 데이터 생성에 유용
- LangChain 프레임워크에서 다양한 종류의 출력 데이터를 파싱하고 처리
### 출력 파서의 이점
- 구조화: LLM의 자유 형식 텍스트 출력을 구조화된 데이터로 변환
- 일관성: 출력 혁식을 일관되게 유지하여 후속 처리 용이
- 유연성: 다양한 출력 형식(JSON, 리스트, 딕셔너리)으로 변환 가능


### 출력 파서를 사용하지 않는 경우
```
**중요 내용 추출:**
1. **발신자:** 김철수 (chulsoo.kim@bikecorporation.me)
2. **수신자:** 이은채 (eunchae@teddyinternational.me)
3. **제목:** "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
4. **요청 사항:**
- ZENESIS 모델의 상세한 브로슈어 요청 (기술 사양, 배터리
성능, 디자인 정보 포함)
5. **미팅 제안:**
- 날짜: 다음 주 화요일 (1월 15일)
- 시간: 오전 10시
- 장소: 귀사 사무실
6. **발신자 정보:**
- 김철수, 상무이사, 바이크코퍼레이션
```

### JSON 형식으로 구조화된 답변
```
{
"person": "김철수",
"email": "chulsoo.kim@bikecorporation.me",
"subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제
안",
"summary": "바이크코퍼레이션의 김철수 상무가 테디인터내셔널
의 이은채 대리에게 신규 자전거 'ZENESIS' 모델에 대한 브로슈어
요청과 기술 사양, 배터리 성능, 디자인 정보 요청. 또한, 협력
논의를 위해 1월 15일 오전 10시에 미팅 제안.",
"date": "1월 15일 오전 10시"
}
```

### PydanticOutputParser 구성요소
- `PydanticOutputParser` (대부분의 OutputParser에 해당)에는 주로 **두 가지 핵심 메서드**가 구현되어야 함
- **`get_format_instructions()`**: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환. 이 지침은 언어 모델이 출력을 구조화하고 특정 데이터 모델에 맞게 변환하는 데 매우 중요.
- **`parse()`**: 언어 모델의 출력(문자열로 가정)을 받아 이를 특정 구조로 분석하고 변환. Pydantic과 같은 도구를 사용하여 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환.
- [Pydantic 공식 도큐먼트](https://docs.pydantic.dev/latest/)


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_teddynote.messages import stream_response

In [3]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

이메일 본문 예시

In [4]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

### 출력 파서를 사용하지 않는 경우

In [5]:
from langchain_core.prompts import PromptTemplate

In [6]:
prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

In [7]:
answer = chain.stream({"email_conversation": email_conversation})
output = stream_response(answer, return_output=True)

이메일의 중요한 내용은 다음과 같습니다:

1. 김철수 상무는 바이크코퍼레이션 소속이며, 이은채 대리에게 이메일을 보냈습니다.
2. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 활동하는 기업입니다.
3. 김철수 상무는 "ZENESIS" 자전거에 대한 상세한 브로슈어를 요청하고 있습니다. 특히 기술 사양, 배터리 성능, 디자인 측면의 정보가 필요합니다.
4. 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안하고 있습니다.
5. 미팅 장소는 이은채 대리의 사무실로 제안되었습니다.

### 출력 파서를 사용하는 경우1: 프롬프트에 포맷 추가하기

데이터 클래스 정의 및 파서 생성

In [8]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [9]:
# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [10]:
# instruction 을 출력
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


프롬프트 정의
1. `question`: 유저의 질문
2. `email_conversation`: 이메일 본문의 내용
3. `format`: 형식 지정

In [11]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [12]:
# 템플릿 출력하고 싶으면 template 사용
print(prompt.template)


You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}



In [13]:
# 템플릿 출력하고 싶으면 template 사용
print(prompt.format(question="test", email_conversation="test_email"))


You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
test

EMAIL CONVERSATION:
test_email

FORMAT:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "stri

chain 생성 및 결과 확인

In [14]:
chain = prompt | llm

In [15]:
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)

```json
{
    "person": "김철수",
    "email": "chulsoo.kim@bikecorporation.me",
    "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
    "summary": "김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.",
    "date": "1월 15일 화요일 오전 10시"
}
```

`parser`를 사용하여 결과를 파싱하고 `EmailSummary` 객체로 변환

In [16]:
# PydanticOutputParser 를 사용하여 결과를 파싱
structured_output = parser.parse(output)
print(structured_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.' date='1월 15일 화요일 오전 10시'


### 출력 파서를 사용하는 경우 2: chain에 parser 추가

In [17]:
parser

PydanticOutputParser(pydantic_object=<class '__main__.EmailSummary'>)

In [18]:
chain = prompt | llm | parser

In [19]:
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.', date='1월 15일 화요일 오전 10시')

### 출력 파서를 사용하는 경우 : .with_structured_output 사용

`.with_structured_output(Pydantic)`을 사용하여 출력 파서를 추가하면, 출력을 Pydantic 객체로 변환함

In [20]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4o"
).with_structured_output(EmailSummary)

In [21]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 이은채 대리에게 바이크코퍼레이션과 테디인터내셔널 간의 협력 가능성을 논의하기 위해 이메일을 보냈습니다. 그는 테디인터내셔널의 새로운 자전거 모델 "ZENESIS"에 대한 관심을 표명하며, 기술 사양, 배터리 성능, 디자인 측면에 대한 상세한 정보를 요청했습니다. 또한, 협력 논의를 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.', date='1월 15일 오전 10시')

그런데 `.with_structured_output()` 함수는 `stream()` 기능을 지원하지 않는다.
왜 stream을 지원하지 않나?   
- with_structured_output()은 LangChain에서 출력 타입을 Pydantic 모델로 변환해주는 유틸리티 함수임
- 내부적으로는 LLM 응답을 한 번에 받아서 이를 Pydantic 모델에 파싱함

- 즉, LLM의 응답이 완전히 끝나야 구조화하여 모델로 변환할 수 있음

